# Exercise 10- Stacking

In this exercise you will implement an ensemble method by learning a stacked regressor.

- paul.kahlmeyer@uni-jena.de

### Submission
- Deadline of submission:
        21.06.23 23:59
- Submission on [moodle page](https://moodle.uni-jena.de/course/view.php?id=43681)


### Help
In case you cannot solve a task, you can use the saved values within the `help` directory:
- Load arrays with [Numpy](https://numpy.org/doc/stable/reference/generated/numpy.load.html)
```
np.load('help/array_name.npy')
```
- Load functions with [Dill](https://dill.readthedocs.io/en/latest/dill.html)
```
import dill
with open('help/some_func.pkl', 'rb') as f:
    func = dill.load(f)
```

to continue working on the other tasks.

# The Dataset

We will use a real world dataset used for predicting the [quality of red wine](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009).
Altough the quality is a discrete value between 0 and 10, we interpret it as a regression task. 

### Task 1

Load the dataset stored in `dataset.csv` and split it into `X` and `y`.

In [149]:
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import BaseEstimator
from sklearn.metrics import r2_score
from sklearn import base

In [150]:
# TODO: load data
# Load the dataset
data = np.loadtxt('dataset.csv', delimiter=',', skiprows=1)

# Split the dataset into features (X) and target variable (y)
X = data[:, :-1]  # All columns except the last one
y = data[:, -1].reshape(-1, 1)  # Last column

# Print the shapes of X and y
print("Features (X) shape:", X.shape)
print("Target variable (y) shape:", y.shape)

Features (X) shape: (1599, 11)
Target variable (y) shape: (1599, 1)


## $R^2$ Score

Sklearn uses the [$R^2$ score](https://en.wikipedia.org/wiki/Coefficient_of_determination) as a quality measure for regressors. Given true values $y$ and predicted values $\hat{y}$ the $R^2$ score is defined as 

\begin{align*}
R^2(y, \hat{y}) &= 1-\cfrac{\sum_{i=1}^m(y_i-\hat{y}_i)^2}{\sum_{i=1}^m(y_i - \bar{y})^2}\,,
\end{align*}
where $\bar{y}$ is the average of $y$.

This value is 1 if the predictions match exactly, 0 if we would simply always predict the average and negative if our predictions are worse than this simple baseline.\
In short we aim for a value $>0$ and close to $1$.

### Task 2

Implement the $R^2$ score.\
Then use scikit learns [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) model to fit on the dataset and calculate the $R^2$ score.\
Compare your result to the `.score` method of the regressor.

In [151]:
def r2_score(y, y_hat):
    '''
    Calculates coefficient of determination.

    @Params:
        y... true labels
        y_hat... predicted labels

    @Returns:
        score in (-inf, 1)
    '''
    return 1 - (np.sum((y - y_hat) ** 2) / np.sum((y - np.mean(y)) ** 2))

# Create and fit the Linear Regression model
regressor = LinearRegression()
regressor.fit(X, y)

# Calculate R2 score using the custom function
y_pred = regressor.predict(X)
r2_custom = r2_score(y, y_pred)

# Calculate R2 score using the score() method of the Linear Regression model
r2_builtin = regressor.score(X, y)

assert(r2_custom == r2_builtin)

print("R2 score:", r2_custom)


R2 score: 0.3605517030386881


# Stacking

The main idea in stacking is to 
1. learn several heterogenous base models on the original data
2. learn a meta model on the predictions of the base models

<div>
<img src="images/stacking.png" width="600"/>
</div>
The hope is that the meta model can learn to combine the strengths of the base models (e.g. if model 1 fails, model 3 is strong).
Note that in contrast to bagging and boosting the base models must not be of the same method (e.g. decision trees).

## Base Models

First lets select a set of base models. We can now choose from the wide pool of regression methods.

Here we want to use the following models:
- [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
- Polynomial Regression of degree 2 (use a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html) of [Polynomial Features](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) followed by Linear Regression)
- [KNN Regression](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
- [Decision Tree Regression](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)


### Task 3
Create a list of base models and evaluate them using crossvalidation (avg. of 10 folds).

In [152]:
# TODO: create base models
# Create a list of base models
base_models = [
    LinearRegression(),
    make_pipeline(PolynomialFeatures(degree=2), LinearRegression()),
    KNeighborsRegressor(),
    DecisionTreeRegressor()
]

# TODO: estimate avg. crossvalidation score for each base model
# Perform cross-validation for each base model
cv_scores = []
for model in base_models:
    scores = cross_val_score(model, X, y, cv=10)
    avg_score = np.mean(scores)
    cv_scores.append(avg_score)

# Print the cross-validation scores for each base model
for i, model in enumerate(base_models):
    print(f"Base Model {i+1}: {cv_scores[i]}")

Base Model 1: 0.2355470969430759
Base Model 2: 0.1902292194320662
Base Model 3: -0.07634595535227609
Base Model 4: -0.5315458182226246


## Meta Model

The meta model uses the predictions of the base models to predict $y$. One can thus view the base models as a feature map for the meta model.

In order to train the meta model, **we need the predictions of the base models on unseen data** since this is the scenario we would face at inference time. A simple method is to use **out-of-fold predictions** during training:

1. separate the data into k folds.
2. hold out one of the folds and train the base models on the other folds.
3. predict the held out fold using the base models.
4. repeat the above two steps k times to obtain out-of-fold predictions for all k folds.
5. feed all the out-of-fold prediction as features (training data) to the meta model.


### Task 4

Implement the out-of-fold method below.\
Calculate the $R^2$-Score on the out-of-fold predictions for each of the base models.

In [153]:
def oof_prediction(model: BaseEstimator, X: np.ndarray, y: np.ndarray, k=5) -> np.ndarray:
    '''
    Calculates out-of-fold predictions.

    @Params:
        model... class with a .fit and .predict method
        X... samples
        y... labels

    @Returns:
        predictions
    '''
    kf = KFold(n_splits=k)
    oof_predictions = np.zeros_like(y)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test).reshape(-1, 1)
        oof_predictions[test_index] = y_pred

    return oof_predictions

# Create a list of base models
base_models = [
    LinearRegression(),
    make_pipeline(PolynomialFeatures(degree=2), LinearRegression()),
    KNeighborsRegressor(),
    DecisionTreeRegressor()
]

# Calculate R2 score for out-of-fold predictions for each base model
for i, model in enumerate(base_models):
    oof_pred = oof_prediction(model, X, y)
    r2_oof = r2_score(y, oof_pred)
    print(f"Base Model {i+1} - R2 Score (Out-of-fold): {r2_oof}")


Base Model 1 - R2 Score (Out-of-fold): 0.33007248077065265
Base Model 2 - R2 Score (Out-of-fold): 0.2788744311971052
Base Model 3 - R2 Score (Out-of-fold): 0.0006190028696213545
Base Model 4 - R2 Score (Out-of-fold): -0.24644357791723825


Now lets put everything together.

### Task 5

Implement the following `Stacking` class. Keep in mind the following things:
- the meta model is trained on out-of-fold predictions of the base models
- the base models are trained on the given dataset
- when predicting, we just use the predictions of the base models (no out-of-fold) as input for the meta model

Use your class to learn a stacked regressor with **linear regression as meta model** and the base models from Task 3. Evaluate it using crossvalidation (avg. of 10 folds) and compare the score to those of the base models (Task 3).

In [154]:
class StackedRegressor(sklearn.base.BaseEstimator):
    
    def __init__(self, base_models : list, meta_model : sklearn.base.BaseEstimator):
        self.base_models = base_models
        self.meta_model = meta_model
        
    def fit(self, X : np.ndarray, y : np.ndarray):
        '''
        Learns base and meta models.
        
        @Params:
            X... features
            y... labels
        '''
        # TODO: implement
        base_pred = np.zeros((X.shape[0], len(self.base_models)))

        for i, model in enumerate(self.base_models):
            model.fit(X,y)
            base_pred[:, i] = oof_prediction(model, X, y).flatten()

        self.meta_model.fit(base_pred, y)
    
    def predict(self, X : np.ndarray) -> np.ndarray:
        '''
        Given features, predicts labels.
        
        @Params:
            X... features
            
        @Returns:
            labels as array
        '''
        # TODO: implement
        base_pred = np.zeros((X.shape[0], len(self.base_models)))

        for i, model in enumerate(self.base_models):
            base_pred[:, i] = model.predict(X).flatten()

        return self.meta_model.predict(base_pred)
    
    
    def score(self, X, y):
        '''
        R2-Score, needed for crossvalidation.
        
        @Params:
            X... features
            y... labels
            
        @Returns:
            Accuracy when predicting for X.
        '''
        # TODO: implement
        y_pred = self.predict(X)
        return r2_score(y, y_pred)

    
# TODO: fit stacked model

# Create the stacked regressor
stacked_regressor = StackedRegressor(base_models=base_models, meta_model=LinearRegression())

# Fit the stacked regressor
stacked_regressor.fit(X, y)

# TODO: evaluate with crossvalidation, compare to base models

# Evaluate with cross-validation and compare to base models
cv_scores_stacked = cross_val_score(stacked_regressor, X, y, cv=10)

print("Cross-validation scores:")
print("Stacked Regressor:", cv_scores_stacked.mean())
for i, model in enumerate(base_models):
    print(f"Base Model {i+1}: {cv_scores[i]}")

Cross-validation scores:
Stacked Regressor: 0.2505126356437329
Base Model 1: 0.2355470969430759
Base Model 2: 0.1902292194320662
Base Model 3: -0.07634595535227609
Base Model 4: -0.5315458182226246


### Task 6

Use the [scikit-learn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html) to learn a stacked regressor.\
Evaluate it using crossvalidation (avg. of 10 folds) and compare the score to the scores of task 5.

Note that minor differences can occur due to a more advanced oof-prediction used in sklearn.

In [155]:
# TODO: fit sklearn stacked model
# Create the stacked regressor
base_models = [
    ('model1', LinearRegression()),
    ('model2', make_pipeline(PolynomialFeatures(degree=2), LinearRegression())),
    ('model3', KNeighborsRegressor()),
    ('model4', DecisionTreeRegressor())
]

stacked_regressor = StackingRegressor(estimators=base_models, final_estimator=LinearRegression())

# TODO: evaluate with crossvalidation, compare to custom model

# Evaluate with cross-validation and compare to base models
cv_scores_stacked_sklearn = cross_val_score(stacked_regressor, X, y.flatten(), cv=10)

print("Cross-validation scores:")
print("Stacked Regressor sklearn:", cv_scores_stacked_sklearn.mean())
print("Stacked Regressor custom:", cv_scores_stacked.mean())

Cross-validation scores:
Stacked Regressor sklearn: 0.24096347646001465
Stacked Regressor custom: 0.2505126356437329


### Task 7
Try at least two different combinations of regressors for base models and meta model and report the average crossvalidation score.
[Here](https://scikit-learn.org/stable/supervised_learning.html) you can find an overview page of sklearn estimators.

In [156]:
# TODO: try different combinations
base_models = [
    [
        ('model1', LinearRegression()),
        ('model2', KNeighborsRegressor()),
        ('model3', Ridge())
    ],
    [
        ('model1', LinearRegression()),
        ('model2', Lasso()),
        ('model3', Ridge())
    ],
    [
        ('model1', Ridge()),
        ('model2', Lasso()),
        ('model3', KNeighborsRegressor())
    ],
    [
        ('model1', Lasso()),
        ('model2', DecisionTreeRegressor()),
        ('model3', KNeighborsRegressor())
    ],
    [
        ('model1', DecisionTreeRegressor()),
        ('model2', LinearRegression()),
        ('model3', KNeighborsRegressor())
    ]
]

meta_models = [
    LinearRegression(),
    make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
]

cv_scores = []
for models in base_models:
    for meta_model in meta_models:
        stacked_regressor = StackingRegressor(estimators=models, final_estimator=meta_model)
        cv_score = cross_val_score(stacked_regressor, X, y.flatten(), cv=10)
        cv_scores.append(np.mean(cv_score))

print(cv_scores)

[0.23759923956468768, 0.2455401883093745, 0.23496884212583816, 0.21432578336503788, 0.23693584293578263, 0.23865227371221548, 0.0819674713573996, 0.046110316683996955, 0.24006837634160755, 0.25186978749763983]
